<a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso-data-management/blob/main/tutorials/03_Upload_clips_to_Zooniverse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #3: Upload clips to Zooniverse</h1>
<h3 align="right">Written by KSO Team</h3>

# Set up KSO requirements

### Install all the requirements

In [ ]:
from IPython.display import clear_output
import os
import sys

try:
    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    IN_COLAB = True
    print("Running in Colab...")

    # Clone repo
    !git clone --recurse-submodules -b master https://github.com/ocean-data-factory-sweden/kso.git
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' kso/yolov5_tracker/requirements.txt) -qr <(sed '/Pillow/d;/ipywidgets/d' kso/yolov5_tracker/yolov5/requirements.txt) -qr <(sed '/Pillow/d;/ipywidgets/d' kso/kso_utils/requirements.txt)

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Navigate to the correct folder
    os.chdir("kso/tutorials")

except:
    IN_COLAB = False


# Ensure widgets are shown properly
!jupyter nbextension enable --user --py widgetsnbextension
!jupyter nbextension enable --user --py jupyter_bbox_widget
!jupyter nbextension enable --user --py ipysheet

clear_output()
if IN_COLAB == True:
    print("Running in Colab: All packages are installed and ready to go!")
else:
    print("Running locally... you're good to go!")

### Import Python packages

In [ ]:
# Set the directory of the libraries
try:
    if "kso_utils" not in sys.modules:
        sys.path.append("..")
        import kso_utils.kso_utils

        sys.modules["kso_utils"] = kso_utils.kso_utils
        print("Using development version...")
        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.widgets as kso_widgets
from kso_utils.project import ProjectProcessor


print("Packages loaded successfully")

### Specify GPU availability

In [ ]:
gpu_available = kso_widgets.gpu_select()

### Choose your project

In [ ]:
project_name = kso_widgets.choose_project()

### Initiate project's database

In [ ]:
# Save the name of the project
project = p_utils.find_project(project_name=project_name.value)

# Initiate pp
pp = ProjectProcessor(project)

### Connect to Zooniverse

In [ ]:
pp.set_zoo_info()

# Select the movie to upload to Zooniverse

In [ ]:
pp.get_movie_info()

### Specify movie of interest

In [ ]:
pp.preview_media()

### Check if movie is already in Zooniverse

In [ ]:
# Remember to query the newest zooniverse data to get the most up to date list of clips uploaded
pp.check_movies_uploaded(pp.movie_selected)

# Create some clip examples (Optional)
Test different parameters (e.g. compression rate, color modifications) in randomly selected clip examples

In [ ]:
pp.generate_zu_clips(
    movie_name=pp.movie_selected,
    movie_path=pp.movie_path,
    is_example=True,
    use_gpu=gpu_available.result,
)

In [ ]:
t_utils.check_clip_size(clips_list=pp.generated_clips.clip_path)

In [ ]:
kso_widgets.compare_clips(
    example_clips=pp.generated_clips.clip_path,
    modified_clips=pp.generated_clips.modif_clip_path,
)

# Create the clips to upload to Zooniverse

In [ ]:
pp.generate_zu_clips(
    movie_name=pp.movie_selected,
    movie_path=pp.movie_path,
    is_example=False,
    use_gpu=gpu_available.result,
)

### Check the size of the clips

In [ ]:
t_utils.check_clip_size(clips_list=pp.generated_clips.clip_path)

# Upload clips to Zooniverse
Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

In [ ]:
pp.upload_zu_subjects("clip")

In [ ]:
# END